# Actor Critic

Метод actor-critic объединяет в себе достоинства методов основанных на политике и на оценке стоимости. Здесь мы оперируем с двумя НС (алгоритмами), Actor и Critic. Actor - аналогичная методу policy gradient нейросеть, за небольшим исключением.  
Метод Policy Gradient, градиент функции :
$$ \nabla L = -\nabla\log(\pi(a_t | s_t))R_t $$
В методе Actor Critic, градиент Actor:
$$ \nabla L_{actor} = -\nabla\log(\pi(a_t | s_t))A_t $$
Где $A_t = Q_t - V_t = r + \gamma V(s') - V(s)$
В свою очередь, значения $V$ оцениваются другой НС, Critic. Градиент Critic:
$$ \nabla L_{critic} = \nabla(r + V(s') - V(s))^2 $$

In [28]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Определяем параметры среды
N = 10  # Длина клеточного мира
num_episodes = 100
gamma = 0.99  # Дисконтирующий фактор

# Определяем актор и критик
class Actor(nn.Module):
    def __init__(self):
        super(Actor, self).__init__()
        self.fc = nn.Linear(1, 2)  # Один вход (состояние), два выхода (действия: влево, вправо)
    
    def forward(self, x):
        return torch.softmax(self.fc(x), dim=-1)

class Critic(nn.Module):
    def __init__(self):
        super(Critic, self).__init__()
        self.fc = nn.Linear(1, 1)  # Один вход, один выход (оценка состояния)
    
    def forward(self, x):
        return self.fc(x)

# Создаем актор и критик
actor = Actor()
critic = Critic()
actor_optimizer = optim.Adam(actor.parameters(), lr=0.01)
critic_optimizer = optim.Adam(critic.parameters(), lr=0.01)

# Функция для выбора действия
def select_action(state):
    state = torch.tensor([[state]], dtype=torch.float32)  # Преобразуем в тензор
    probs = actor(state)  # Получаем вероятности действий
    action = np.random.choice(2, p=probs.detach().numpy()[0])  # Выбираем действие по вероятностям
    return action

# Обучение агента
for episode in range(num_episodes):
    state = 0  # Начальная позиция
    done = False
    
    while not done:
        action = select_action(state)  # Выбор действия
        
        if action == 0:  # Действие: влево
            next_state = max(0, state - 1)
        else:  # Действие: вправо
            next_state = min(N - 1, state + 1)
        
        reward = 1.0 if next_state == N - 1 else 0.0  # Награда
        
        # Обучаем критика
        state_tensor = torch.tensor([[state]], dtype=torch.float32)
        next_state_tensor = torch.tensor([[next_state]], dtype=torch.float32)
        
        value = critic(state_tensor)
        next_value = critic(next_state_tensor)
        target = reward + gamma * next_value
        
        critic_loss = (value - target.detach()).pow(2).mean()
        
        critic_optimizer.zero_grad()
        critic_loss.backward()
        critic_optimizer.step()
        
        # Обучаем актора
        advantage = (target - value).detach()
        
        probs = actor(state_tensor)
        action_prob = probs[0, action]
        actor_loss = -torch.log(action_prob) * advantage
        
        actor_optimizer.zero_grad()
        actor_loss.backward()
        actor_optimizer.step()
        
        state = next_state
        
        if state == N - 1:  # Если достигли терминальной клетки
            done = True

print("Обучение завершено!")


Обучение завершено!


Стратегия в произвольном состоянии

In [31]:
probs = actor(torch.tensor([[6]], dtype=torch.float32))
probs

tensor([[1.5365e-06, 1.0000e+00]], grad_fn=<SoftmaxBackward0>)